## 1. read data from MongoDB, store it into JSON data

In [82]:
import pymongo
import json

In [83]:
def mongo_connect_build(db_name, col_name):
    
    global mycol
    client = pymongo.MongoClient("mongodb://192.168.1.25:27017/") # 跟mongodb建立連

    db = client[db_name] # 選擇使用的db,不存在則會在資料輸入時自動建立
    mycol = db[col_name] # 選擇collection,不存在則會在資料輸入時自動建立
    
def data_find():
    # 尋找多筆資料
    return mycol.find({}, {"_id":0})

In [84]:
# # connect to mongoDB to get data
# mongo_connect_build('Topic_104', 'job_clean_test') 
# lst_jobdata = [i for i in data_find()] # store the data

In [85]:
def open_json_file(CACHE_FNAME):
    try:
        cache_file = open(CACHE_FNAME, 'r', encoding='utf-8-sig')
        cache_contents = cache_file.read()
        CACHE_DICTION = json.loads(cache_contents, encoding='utf-8-sig')
        cache_file.close()
        return CACHE_DICTION

    except:
        CACHE_DICTION = {}
        return CACHE_DICTION
    
def dump_json_file(query_dict, file_name):
    dumped_json_cache = json.dumps(query_dict)
    fw = open(file_name,"w")
    fw.write(dumped_json_cache)
    fw.close()
    print('successfully write down the file: ', file_name)

In [86]:
# dump_json_file(lst_jobdata, 'json_data/test_NLP_01.json')

In [87]:
# open the data that alreday store in local json file

job_data = open_json_file('json_data/test_NLP_01.json')

In [88]:
print('所有資料數量: ', len(job_data))

所有資料數量:  135384


## 2. Clean job Description data

In [89]:
import re

### a. 定義資料清洗function

In [90]:
def isEnglish(s): # 檢查字元是否為英文
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
def isNumber(s): # 檢查字元是否為數字
    return s.isdigit()



In [ ]:
#將只有中文名,其他全英文字的履歷刪除
myquery = {"value": {"$not":{"$regex":"[\u4e00-\u9fa5]{4}[.]*[\u4e00-\u9fa5]+"}}}
myquery = {"value": {"$not":{"$regex":"[\u4e00-\u9fa5]{3}[.]*[\u4e00-\u9fa5]+"}}}
myquery = {"value": {"$not":{"$regex":"[\u4e00-\u9fa5]{2}[.]*[\u4e00-\u9fa5]+"}}}
myquery = {"value": {"$not":{"$regex":"[\u4e00-\u9fa5]{1}[.]*[\u4e00-\u9fa5]+"}}}

In [91]:
# show original data
for i in job_data[0:2]:
    print(i['jobDescription'])
    print(i['other'])
    print("-"*20)

1.負責介紹及銷售商品。
2.提供顧客之接待與需求服務（如：電話諮詢、調貨、修改、包裝及退換貨處理）。
3.負責商品進貨入庫、銷售管理及庫存管理。
4.負責商品包裝、陳列及促銷品換檔工作。
5.維持店櫃週遭整潔。
6.提升業績。
需符合以下條件，無經驗勿投履歷~
1.具備少淑女服飾一年以上銷售經驗
2.充滿活力並有服務熱忱
--------------------
歐德傢俱連鎖事業鼓勵對系統傢俱設計有濃厚的有志之士加入，透過完整的晉升制度及各項多元職系發展，讓同仁在歐德集團開創自己的事業。
本職務有業務經驗尤佳，以設計、銷售系統傢俱為主，依業主需求從系統介紹、工地丈量、繪圖、監工等..為業主服務，讓您走向專業發展的設計領域，歡迎您加入歐德集團。
自備交通工具
--------------------


In [92]:
def string_clean(jobDescription): 
    '''
    input: string (original job description)
    output: string (clean job description)
    '''
    job_desc = jobDescription.split('\r\n') # 根據換行符號轉乘 List格式
    job_words = ''
    
    for words in job_desc:
        words = re.sub(r'[^\w\s]',' ', words)  # remove all punctuations
        words = re.sub(r'\d+', '', words)  # remove all numbers
        words = words.strip() # remove white space
        job_words += words
    
    return (job_words)

In [93]:
# show data after cleaning
for i in job_data[0:2]:
    jobDescription = i['jobDescription']
    jobOther = i['other']
    
    job_words = string_clean(jobDescription) # clean string
    job_words_other = string_clean(jobOther) # clean string

    jobDescription_clean = job_words + job_words_other # add two string
    
#     if isEnglish(jobDescription_clean) == True:
#         print(jobDescription_clean)
 
    print(jobDescription_clean)
    print("-"*20)

負責介紹及銷售商品提供顧客之接待與需求服務 如 電話諮詢 調貨 修改 包裝及退換貨處理負責商品進貨入庫 銷售管理及庫存管理負責商品包裝 陳列及促銷品換檔工作維持店櫃週遭整潔提升業績需符合以下條件 無經驗勿投履歷具備少淑女服飾一年以上銷售經驗充滿活力並有服務熱忱
--------------------
歐德傢俱連鎖事業鼓勵對系統傢俱設計有濃厚的有志之士加入 透過完整的晉升制度及各項多元職系發展 讓同仁在歐德集團開創自己的事業本職務有業務經驗尤佳 以設計 銷售系統傢俱為主 依業主需求從系統介紹 工地丈量 繪圖 監工等  為業主服務 讓您走向專業發展的設計領域 歡迎您加入歐德集團自備交通工具
--------------------


#### 資料清洗整合

In [94]:
# 將不同欄位的資料清洗並整合，並排除英文職缺

def concate_clean_data(data):
    jobDescription = data['jobDescription']
    jobOther = data['other']
    
    job_words = string_clean(jobDescription) # clean string
    job_words_other = string_clean(jobOther) # clean string

    jobDescription_clean = job_words + job_words_other # add two string
    
    # 如果是英文職缺，回傳 None
    if isEnglish(jobDescription_clean) == True:
        return None
    
    # add jobCategory
    jobDescription_clean = jobDescription_clean + " " + data['jobCategory'].replace('／', ' ')
    
    # add major_clean
    if len(data['major_clean']) > 0:
        for j in data['major_clean']:
            jobDescription_clean = jobDescription_clean + " " + j
    
    # add language_clean
    if len(data['language_clean']) > 0:
        for j in data['language_clean']:
            jobDescription_clean = jobDescription_clean + " " + j
    
    # add skill_clean
    if len(data['skill_clean']) > 0:
        for j in data['skill_clean']:
            jobDescription_clean = jobDescription_clean + " " + j
            
    # add certificate_clean
    if len(data['certificate_clean']) > 0:
        for j in data['certificate_clean']:
            jobDescription_clean = jobDescription_clean + " " + j

    # add specialty
    if len(data['specialty']) > 0:
        for j in data['specialty']:
            jobDescription_clean = jobDescription_clean + " " + j
            
    return jobDescription_clean
    

In [95]:
for i in job_data[0:3]:
    data = concate_clean_data(i)
    if data != None:
        print(data)
        print("-"*20)

負責介紹及銷售商品提供顧客之接待與需求服務 如 電話諮詢 調貨 修改 包裝及退換貨處理負責商品進貨入庫 銷售管理及庫存管理負責商品包裝 陳列及促銷品換檔工作維持店櫃週遭整潔提升業績需符合以下條件 無經驗勿投履歷具備少淑女服飾一年以上銷售經驗充滿活力並有服務熱忱 門市 店員 專櫃人員 Outlook
--------------------
歐德傢俱連鎖事業鼓勵對系統傢俱設計有濃厚的有志之士加入 透過完整的晉升制度及各項多元職系發展 讓同仁在歐德集團開創自己的事業本職務有業務經驗尤佳 以設計 銷售系統傢俱為主 依業主需求從系統介紹 工地丈量 繪圖 監工等  為業主服務 讓您走向專業發展的設計領域 歡迎您加入歐德集團自備交通工具 國內業務人員 客戶資料更新維護 業務或通路開發 業績目標分配與績效達成 Word
--------------------
熟悉成本會計熟悉三角貿易會計進貨 銷貨 費用單據審核及傳票製作 零用金異動及撥補應收 應付帳務處理 會計報表編製 有操作系統經驗者佳採購與庫存管理 進銷存管理 與廠商接洽訂單事宜須熟悉正航會計軟體能完成財務報表公司福利週休二日特休 年假依勞基法規定須熟悉正航會計軟體 記帳 出納 一般會計 會計學 一般商業 英文 Adobe Photoshop
--------------------


### b. 使用 Jieba 分詞

In [96]:
import jieba

#### 自定義分詞用字典檔(千萬不要跑這邊的代碼!!!!!)

In [97]:
# 千萬不要跑這邊的代碼!!!!!

# # 設定自定義的詞庫，寫入 'jieba_data/Jobcontent_dict.txt'

# set_dict = []

# # 將 major的字彙匯入字典
# for i in job_data:
#     if len(i['major_clean']) > 0:
#         for major in i['major_clean']:
#             if major not in set_dict:
#                 set_dict.append(major)

# # 將 language 的字彙匯入字典
# for i in job_data:
#     if len(i['language_clean']) > 0:
#         for language in i['language_clean']:
#             if language not in set_dict:
#                 set_dict.append(language)

                
# # 將 certificate 的字彙匯入字典
# for i in job_data:
#     if len(i['certificate_clean']) > 0:
#         for certificate in i['certificate_clean']:
#             if certificate not in set_dict:
#                 set_dict.append(certificate)


# # 將 specialty 的字彙匯入字典
# for i in job_data:
#     if len(i['specialty']) > 0:
#         for certificate in i['specialty']:
#             certificate = certificate.strip()
#             if certificate not in set_dict:
#                 set_dict.append(certificate)


# with open(file='jieba_data/Jobcontent_dict.txt',mode='a', encoding="UTF-8") as writer:
#     for w in set_dict:
#         writer.write(w + '\n')


<span style="color:red"> 自定義字典檔整合 (小組合作) </span>

In [98]:
# 計算職缺分類與數量

dict_cat = {}

for i in job_data:
    jobCat_main = i['jobCat_main']
    if jobCat_main not in dict_cat.keys():
        dict_cat[jobCat_main] = 1
    else:
        dict_cat[jobCat_main] += 1
        
print(dict_cat)

# 01_Vivian: '客服╱門市╱業務╱貿易類'
# 02_炫斐: '餐飲╱旅遊 ╱美容美髮類',  '學術╱教育╱輔導類'
# 03_韋智: '財會╱金融專業類', '操作╱技術╱維修類','生產製造╱品管╱環衛類'
# 04_謙恆: '經營╱人資類', '文字╱傳媒工作類', '資訊軟體系統類', '研發相關類'
# 05_Willy: '資材╱物流╱運輸類', '醫療╱保健服務類', '行銷╱企劃╱專案管理類'
# 06_董哥: '行政╱總務╱法務類', '傳播藝術╱設計類', '軍警消╱保全類', '營建╱製圖類'

{'客服╱門市╱業務╱貿易類': 25650, '財會╱金融專業類': 5404, '操作╱技術╱維修類': 11187, '餐飲╱旅遊 ╱美容美髮類': 20486, '生產製造╱品管╱環衛類': 8451, '經營╱人資類': 3293, '文字╱傳媒工作類': 684, '資訊軟體系統類': 9405, '資材╱物流╱運輸類': 5419, '醫療╱保健服務類': 5511, '行銷╱企劃╱專案管理類': 7421, '研發相關類': 6940, '行政╱總務╱法務類': 6479, '傳播藝術╱設計類': 3616, '軍警消╱保全類': 2399, '營建╱製圖類': 6009, '學術╱教育╱輔導類': 5060, '其他職類': 1970}


In [99]:
# 自定義字典檔整合: 請先下載所有的字典檔

def txt_file_integration():
    lst_jieba_dict = []

    for i in range(1,7):
        file_name = 'jieba_data/Jobcontent_dict_0' + str(i) + '.txt'
        print('read file: ', file_name)
        with open(file=file_name , mode='r', encoding="UTF-8") as file:
            words = file.read().split('\n') 
            if len(words) > 0:
                for i in words:
                    if i not in lst_jieba_dict:
                        lst_jieba_dict.append(i)

    with open(file='jieba_data/Jobcontent_dict.txt' , mode='a', encoding="UTF-8") as writer:
        for w in lst_jieba_dict:
            writer.write(w + '\n')

    print('Successful append data into "jieba_data/Jobcontent_dict.txt" ' )

In [100]:
# 執行txt資料整合
txt_file_integration()

read file:  jieba_data/Jobcontent_dict_01.txt
read file:  jieba_data/Jobcontent_dict_02.txt
read file:  jieba_data/Jobcontent_dict_03.txt
read file:  jieba_data/Jobcontent_dict_04.txt
read file:  jieba_data/Jobcontent_dict_05.txt
read file:  jieba_data/Jobcontent_dict_06.txt
Successful append data into "jieba_data/Jobcontent_dict.txt" 


<span style="color:red"> 自定義字典檔去掉重複值 </span>

In [101]:
def remove_duplicate(textfile):

    lst_word = []

    with open(file=textfile , mode='r', encoding="UTF-8") as file:
        words = file.read().split('\n') 
        for i in words:
            if i not in lst_word:
                lst_word.append(i)

    with open(file=textfile , mode='w', encoding="UTF-8") as writer:
        for w in lst_word:
            writer.write(w + '\n')

In [102]:
# 自定義字典檔去掉重複值

remove_duplicate('jieba_data/Jobcontent_dict.txt')

<span style="color:red"> 使用 Jieba 分詞 </span>

In [150]:
# 指定辭典檔
jieba.load_userdict('jieba_data/Jobcontent_dict.txt')


In [157]:
# 測試斷詞用

# for i in job_data[1000:5000]:
#     if i['jobCat_main'] == '客服╱門市╱業務╱貿易類':
#         data = concate_clean_data(i)
#         if data != None:
#             print(data)
#             print("-"*10)
#             # 使用結巴斷詞
#             seg_result = jieba.cut(data, cut_all=False)
#             str_result = '/'.join([i for i in list(seg_result) if len(i) > 0])
#             print(str_result)
#             print("="*30)

In [153]:
# 使用結巴斷詞，產生 list of words

def jieba_cut(data):
    
    # 使用結巴斷詞
    seg_result = jieba.cut(data, cut_all=False)

    # 篩選斷詞，去掉單一中文字
    lst_seg = []
    
    for i in list(seg_result):
        i = i.strip()
        if len(i) < 1:  #排除空值
            continue
        elif i in stop_words: # 排除stopwords
            continue
        elif isEnglish(i) == False and len(i) == 1: #排除單一中文字
            continue
        elif isNumber(i) == True: #排除數字
            continue
        else:
            lst_seg.append(i)
   
    return lst_seg

# 建立stopwords用

In [152]:
# 排除字元表單 stopword, 開啟 'Jobcontent_stopwords.txt'檔案

with open(file='jieba_data/Jobcontent_stopwords.txt',mode='r', encoding="UTF-8") as file:
    stop_words = file.read().split('\n')
    stop_words = [i.strip() for i in stop_words]


In [158]:
# # 建立stopwords用

# dict_words_frequency = {}

# for i in job_data:
#     data = concate_clean_data(i)
#     if data != None:
        
#         # 使用結巴斷詞
#         lst_seg = jieba_cut(data)
        
#         # 計算斷詞出現頻率
#         for i in lst_seg:
#             if i not in dict_words_frequency:
#                 dict_words_frequency[i] = 1
#             else:
#                 dict_words_frequency[i] += 1

# dict_words_frequency = {k: v for k, v in sorted(dict_words_frequency.items(), key=lambda item: item[1], reverse=True)}

# # print(dict_words_frequency) 

# # 已篩選大於500頻率字
# for k, v in dict_words_frequency.items():
#     if v < 500 and v > 100:
#         print(k)


In [156]:
# 顯示使用stopwords之後的斷詞結果

for i in job_data[0:2]:
    data = concate_clean_data(i)
    if data != None:
        
        # 使用結巴斷詞
        lst_seg = jieba_cut(data)
        
        # 顯示斷詞結果    
        str_result = '/'.join(lst_seg)
        print(str_result)
        print("="*30)

介紹/銷售/商品/顧客/接待/電話/諮詢/調貨/包裝/退換貨/商品/進貨/入庫/銷售/管理/庫存/管理/商品/包裝/陳列/促銷品/換檔/維持/店櫃/週遭/整潔/提升/業績/淑女/服飾/銷售/門市/店員/專櫃/Outlook
傢俱/連鎖/事業/系統/傢俱/設計/有志之士/多元/職系/發展/同仁/開創/事業/業務/設計/銷售/系統/傢俱/業主/系統/介紹/工地/丈量/繪圖/監工/業主/走向/專業/發展/設計/領域/交通工具/國內/業務人員/客戶/資料/維護/業務/通路/開發/業績/分配/Word
